In [1]:
import pandas as pd
from gensim import corpora, models
import pandas as pd
import numpy as np
import sys
from collections import Counter
sys.path.append("c:/users/david/desktop/sandbox/lsi-tagger")
from model.tag_extractor import TagExtractor
from model.text_cleaner import TextCleaner
import itertools

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('data/h&m_kaggle_products.csv')
df = df[~df.isnull().any(axis=1)]
print(df.shape)
df.head(2)

(105126, 25)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [3]:
documents = df['detail_desc'].drop_duplicates().values
cleaned_documents = TextCleaner(word_count_min=2, word_length_min=2).clean_documents(documents)
len(cleaned_documents)

100%|█████████████████████████████████████████████████████████████████████████| 43404/43404 [00:04<00:00, 10115.35it/s]


43404

In [4]:
# Remove empty documents
doc_keep_inds = np.array([n for n,doc in enumerate(cleaned_documents) if len(doc)>0])
cleaned_documents = np.array(cleaned_documents)[doc_keep_inds].tolist()
documents = np.array(documents)[doc_keep_inds].tolist()

len(documents), len(cleaned_documents)

C:\Users\David\AppData\Local\Temp\ipykernel_15636\2820098043.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cleaned_documents = np.array(cleaned_documents)[doc_keep_inds].tolist()


(43404, 43404)

In [5]:
# LSI (https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html)
  # https://radimrehurek.com/gensim/models/lsimodel.html
dictionary = corpora.Dictionary(cleaned_documents)
corpus = [dictionary.doc2bow(doc) for doc in cleaned_documents]

len(dictionary)

2003

In [6]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=200)
corpus_lsi = lsi_model[corpus_tfidf]

In [7]:
lsi_model.print_topics(5)

[(0,
  '0.179*"long" + 0.178*"sleeves" + 0.167*"soft" + 0.165*"cuffs" + 0.165*"short" + 0.163*"hem" + 0.161*"pockets" + 0.158*"cotton" + 0.152*"jersey" + 0.148*"waist"'),
 (1,
  '-0.233*"pockets" + 0.232*"knit" + 0.223*"ribbing" + 0.211*"jumper" + -0.207*"fly" + 0.198*"neckline" + 0.183*"long" + -0.174*"legs" + 0.174*"hem" + 0.164*"cuffs"'),
 (2,
  '0.234*"dress" + 0.206*"seam" + -0.204*"ribbing" + 0.194*"neck" + -0.189*"pockets" + 0.179*"short" + -0.178*"sweatshirt" + 0.165*"skirt" + 0.151*"unlined" + 0.149*"weave"'),
 (3,
  '-0.331*"imitation" + -0.323*"leather" + -0.289*"insoles" + -0.288*"soles" + -0.258*"linings" + -0.237*"cm" + -0.220*"rubber" + -0.196*"loop" + -0.155*"heel" + -0.141*"lacing"'),
 (4,
  '0.268*"jersey" + -0.267*"collar" + -0.223*"buttons" + 0.203*"elasticated" + -0.193*"buttoned" + -0.183*"chest" + -0.183*"jacket" + -0.162*"flap" + 0.160*"soft" + -0.155*"rounded"')]

In [8]:
def lsi_corpus2vec(lsi_corpus_vec):
    vec = np.zeros(200)
    for ind, value in lsi_corpus_vec:
        vec[ind] = value
    return vec

lsi_vecs = np.array([lsi_corpus2vec(cl) for cl in corpus_lsi])
lsi_vecs_normed = lsi_vecs/np.linalg.norm(lsi_vecs, axis=1)[:,None]

In [9]:
lsi_topic_matrix = lsi_model.get_topics()

def extract_tags(input_ind, candidate_ind, n_tags=5):

    # Get shared words using the sparse TF-IDF corpus
    shared_word_ids = np.array(list(set(np.array(corpus_tfidf[input_ind])[:,0]) \
                                  & set(np.array(corpus_tfidf[candidate_ind])[:,0]))).astype(int)

    # Get word-topic vectors for those shared words
    shared_word_topics = lsi_topic_matrix.T[shared_word_ids]

    # Sum those word-topic vectors weighed by each document's topic loadings
    shared_lsi_word_loadings = np.sum((shared_word_topics*np.array(corpus_lsi[input_ind])[:,1]) \
                                    + (shared_word_topics*np.array(corpus_lsi[candidate_ind])[:,1]), axis=1)

    # Rank shared words by highest weighed word-topic loadings
    ranking = np.argsort(-shared_lsi_word_loadings)
    ranked_shared_word_ids = shared_word_ids[ranking]

    tags = [dictionary[r] for r in ranked_shared_word_ids][:n_tags]
    scores = shared_lsi_word_loadings[ranking][:n_tags]
    return tags, scores

def show_recommendations_with_tags(input_ind, internal_n_recs=50, show_n_recs=5,
                                   n_input_tags=10, n_candidate_tags=5):
    sims = lsi_vecs_normed[input_ind].dot(lsi_vecs_normed.T)
    top_inds = np.argsort(-sims)[:internal_n_recs]

    results = []
    all_tags = []
    for candidate_ind in top_inds:
        tags, scores = extract_tags(input_ind, candidate_ind, n_tags=n_candidate_tags)
        results.append([candidate_ind, tags, scores])
        all_tags.extend(tags)

    print(documents[input_ind])
    print(Counter(all_tags).most_common(n_input_tags))
    print('='*150)
    print()

    for data in results[:show_n_recs]:
        print(documents[data[0]])
        display(list(zip(data[1], data[2])))
        print('-'*150)

In [10]:
# TODO: Ability to re-rank by selecting tags
show_recommendations_with_tags(10000, internal_n_recs=50, show_n_recs=5,
                               n_input_tags=10, n_candidate_tags=5)

Straight-cut, linen-blend jacket in a patterned weave with a shawl collar, padded shoulders and long sleeves with decorative buttons at the cuffs. Buttons at the front and front pockets. Lined.
[('buttons', 43), ('cut', 35), ('straight', 35), ('linen', 32), ('jacket', 31), ('decorative', 18), ('blend', 12), ('collar', 10), ('weave', 9), ('shoulders', 7)]

Straight-cut, linen-blend jacket in a patterned weave with a shawl collar, padded shoulders and long sleeves with decorative buttons at the cuffs. Buttons at the front and front pockets. Lined.


[('buttons', 0.6640270897830053),
 ('linen', 0.6474020570456075),
 ('cut', 0.48276344712755204),
 ('straight', 0.4279763355143735),
 ('decorative', 0.42295154709549243)]

------------------------------------------------------------------------------------------------------------------------------------------------------
Straight-cut jacket in a patterned linen and viscose weave with notch lapels, buttons at the front, welt front pockets and decorative buttons at the cuffs. Partly lined.


[('buttons', 0.6663840119373068),
 ('linen', 0.6460366893667935),
 ('cut', 0.4762145099332802),
 ('jacket', 0.4514144555624118),
 ('straight', 0.43481499143204294)]

------------------------------------------------------------------------------------------------------------------------------------------------------
Straight-cut, double-breasted jacket in a patterned weave with notch lapels and covered buttons at the front. Long sleeves with decorative buttons at the cuffs and welt front pockets with a flap. Lined.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
Straight-cut jacket in woven fabric with a shawl collar, thin shoulder pads, jetted front pockets and no buttons. Lined.


[('buttons', 0.501594116934601),
 ('cut', 0.48387922628842),
 ('straight', 0.43169388781843715),
 ('jacket', 0.4023984568715535),
 ('collar', 0.2646230191469885)]

------------------------------------------------------------------------------------------------------------------------------------------------------
Linen-blend jacket in a herringbone weave with notch lapels, a covered button at the front, welt front pockets with a flap, and decorative buttons at the cuffs. Lined.


[('linen', 0.6370003612129868),
 ('buttons', 0.510355252164045),
 ('jacket', 0.421569619197373),
 ('decorative', 0.39685611634302975),
 ('blend', 0.3613195647872708)]

------------------------------------------------------------------------------------------------------------------------------------------------------


In [46]:
lsi_topic_matrix_T_normed = lsi_topic_matrix.T/np.linalg.norm(lsi_topic_matrix.T, axis=1)[:,None]

In [88]:
# TODO: Can give the user options of adjacent tags so that they can get new queries entirely?
# They can remove tags from the input document and add in other tags, like swapping out pants for jeans, etc.
# This results in a "new" input document

tags = cleaned_documents[777]
print(tags)
bow = dictionary.doc2bow(tags)
bow_vec = lsi_corpus2vec(lsi_model[tfidf[bow]])
bow_vec_normed = bow_vec/np.linalg.norm(bow_vec)
sims = bow_vec_normed.dot(lsi_topic_matrix_T_normed.T)
top_inds = np.argsort(-sims)
count = 0
for t in top_inds:
    tag = dictionary[t]
    if tag not in tags:
        print(tag)
        count += 1
    if count == 5:
        break

['single', 'breasted', 'jacket', 'stretch', 'weave', 'containing', 'wool', 'narrow', 'notch', 'lapels', 'decorative', 'buttonhole', 'fastening', 'buttons', 'chest', 'pocket', 'flap', 'pockets', 'inner', 'pockets', 'button', 'decorative', 'buttons', 'cuffs', 'single', 'vent', 'lined', 'skinny', 'fit', 'slightly', 'shorter', 'style', 'shapes', 'chest', 'tapers', 'sharply', 'waist', 'combined', 'narrow', 'shoulders', 'sleeves', 'creates', 'completely', 'fitted', 'silhouette']
narrower
tuxedo
linked
special
search


In [156]:
class TagExtractor:
    def __init__(self, 
                 word_count_min=2, 
                 word_length_min=2, 
                 num_lsi_topics=300):
        self.word_count_min = word_count_min
        self.word_length_min = word_length_min
        self.num_lsi_topics = num_lsi_topics
    
    def fit(self, documents):
        # Clean text
        # TODO: TextCleaner needs a fit and transform method so that I can look up word counts for new documents
        self.tc = TextCleaner(word_count_min=self.word_count_min, 
                              word_length_min=self.word_length_min)
        cleaned_documents = self.tc.clean_documents(documents)
            
        # Create document lookup
        self.problem_docs = []
        self.doc2ind = {}
        for n, (doc, cleaned_doc) in enumerate(zip(documents, cleaned_documents)):
            if len(cleaned_doc)==0:
                import pdb; pdb.set_trace()
                self.problem_docs.append(doc)
            self.doc2ind[doc] = n
                
        # Warn for empty documents
        if len(self.problem_docs) > 0:
            print("""Warning: Some documents yield no clean tokens. These documents won't have tags. Check self.problem_docs for more detail.""")
        
        # Train TF-IDF
        self.dictionary = corpora.Dictionary(cleaned_documents)
        self.corpus = [self.dictionary.doc2bow(doc) for doc in cleaned_documents]
        self.tfidf = models.TfidfModel(self.corpus)
        self.corpus_tfidf = self.tfidf[self.corpus]

        # Train LSI
        self.lsi_model = models.LsiModel(self.corpus_tfidf, 
                                         id2word=self.dictionary, 
                                         num_topics=self.num_lsi_topics)
        self.corpus_lsi = self.lsi_model[self.corpus_tfidf]
        
        # Save the topic matrix for tag extraction
        self.lsi_topic_matrix = self.lsi_model.get_topics()
        
    def _transform(self, document):
        doc_ind = self.doc2ind.get(document) 
        if doc_ind is None:
            # TODO: TextCleaner needs a fit and transform method so that I can look up word counts for new documents
            cleaned_documents = self.tc.clean_documents([document], display_progress=False)
            corpus = self.dictionary.doc2bow(cleaned_documents[0])
            corpus_tfidf = self.tfidf[corpus]
            corpus_lsi = self.lsi_model[corpus_tfidf]
        else:
            corpus_tfidf = self.corpus_tfidf[doc_ind]
            corpus_lsi = self.corpus_lsi[doc_ind]
        return corpus_tfidf, corpus_lsi
    
    def extract_tags_and_rank(self, input_document, ranked_candidate_documents, 
                              n_input_tags, n_candidate_tags, selected_tags=[]):
        tfidf_input, lsi_input = self._transform(input_document)
        if (len(tfidf_input) == 0) | (len(lsi_input) == 0):
            return [], [], []
        
        tag_scores = []
        for candidate_document in ranked_candidate_documents:
            tfidf_candidate, lsi_candidate = self._transform(candidate_document)
            
            if (len(tfidf_candidate) == 0) | (len(lsi_candidate) == 0):
                tag_scores.append({})
            else:

                # Get shared words using the sparse TF-IDF corpus
                shared_word_ids = np.array(list(set(np.array(tfidf_input)[:,0]) \
                                              & set(np.array(tfidf_candidate)[:,0]))).astype(int)

                # Get word-topic vectors for those shared words
                shared_word_topics = self.lsi_topic_matrix.T[shared_word_ids]

                # Sum those word-topic vectors weighed by each document's topic loadings
                shared_lsi_word_loadings = np.sum((shared_word_topics*np.array(lsi_input)[:,1]) \
                                                + (shared_word_topics*np.array(lsi_candidate)[:,1]), axis=1)

                # Rank shared words by highest weighed word-topic loadings
                ranking = np.argsort(shared_lsi_word_loadings)[::-1]
                ranked_shared_word_ids = shared_word_ids[ranking]

                tags = [self.dictionary[r] for r in ranked_shared_word_ids][:n_candidate_tags]
                scores = shared_lsi_word_loadings[ranking][:n_candidate_tags].tolist()

                tag_scores.append(dict(zip(tags, scores)))
            
        all_candidate_tags = list(itertools.chain(*[list(ts.keys()) for ts in tag_scores]))
        input_tags = Counter(all_candidate_tags).most_common(n_input_tags)
        
        if len(selected_tags) > 0:
            reranking_scores = []
            for tag_dict in tag_scores:
                document_score = sum(tag_dict.get(tag,0) for tag in selected_tags)
                reranking_scores.append(document_score)
            ranking = np.argsort(-np.array(reranking_scores)).tolist()
        else:
            ranking = list(range(len(ranked_candidate_documents)))
            
        return input_tags, tag_scores, ranking

In [157]:
te = TagExtractor(word_count_min=1, word_length_min=2, num_lsi_topics=200)
te.fit(df['detail_desc'].drop_duplicates().values)

100%|██████████████████████████████████████████████████████████████████████████| 43404/43404 [00:04<00:00, 9508.30it/s]


In [158]:
df['detail_desc'].drop_duplicates().values[:5]

array(['Jersey top with narrow shoulder straps.',
       'Microfibre T-shirt bra with underwired, moulded, lightly padded cups that shape the bust and provide good support. Narrow adjustable shoulder straps and a narrow hook-and-eye fastening at the back. Without visible seams for greater comfort.',
       'Semi shiny nylon stockings with a wide, reinforced trim at the top. Use with a suspender belt. 20 denier.',
       'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
       'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.'],
      dtype=object)

In [159]:
te.extract_tags_and_rank(input_document='Jersey top with narrow shoulder straps.', 
                         ranked_candidate_documents=[
                             'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                             'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                             'Jersey top with narrow shoulder straps.'
                          ], 
                          n_input_tags=10, n_candidate_tags=5, 
                          selected_tags=[])

([('narrow', 1), ('shoulder', 1), ('straps', 1), ('jersey', 1)],
 [{},
  {},
  {'narrow': 1.0847161183274086,
   'shoulder': 1.0523107110568781,
   'straps': 0.9339358005822074,
   'jersey': 0.5775927072817127}],
 [0, 1, 2])

In [160]:
te.extract_tags_and_rank(input_document='Jersey top with narrow shoulder straps.', 
                         ranked_candidate_documents=[
                             'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                             'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                             'Jersey top with narrow shoulder straps.'
                          ], 
                          n_input_tags=10, n_candidate_tags=5, 
                          selected_tags=['narrow'])

([('narrow', 1), ('shoulder', 1), ('straps', 1), ('jersey', 1)],
 [{},
  {},
  {'narrow': 1.0847161183274086,
   'shoulder': 1.0523107110568781,
   'straps': 0.9339358005822074,
   'jersey': 0.5775927072817127}],
 [2, 0, 1])